In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
item= pd.read_csv('../main/datasets/1.0v/items.csv', sep = '|')
info = pd.read_csv('../main/datasets/1.0v/infos.csv', sep = '|')
order= pd.read_csv('../main/datasets/1.0v/orders.csv', sep = '|')

In [4]:
#deixando apenas com promocoes
info_promotion = info[info.promotion.notna()]
print(info_promotion.min().promotion)
print(info_promotion.max().promotion)

2018-06-30
2018-07-13


todas as promoçoes acontecem depois de 6/30, ou seja temos APENAS informaçoes de promoçoes que vao acontecer no periodo que temos q prever as vendas

In [5]:
#adicionando coluna recommendedRetailPrice a order

RetailPrice = item[['itemID','recommendedRetailPrice']]
order = pd.merge(order, RetailPrice, on='itemID')

print(len(order[order.salesPrice == order.recommendedRetailPrice]))
print(len(order[order.salesPrice != order.recommendedRetailPrice]))

8321
2173634


grande parte das vendas n~ao utiliza preço recomendado pelo retail

### **adicionando a feature: moda de cada produto(itemID)**

In [6]:
def get_mode_sales_price(id):
  sales_id_certo = order[order['itemID'] == id]
  mode = sales_id_certo['salesPrice'].mode()

  #a moda pode ter mais de um valor, 
  #para torna-la um unico foi escolhido arbitrariamente a mediana
  return mode.median()

In [7]:
#adicionando a moda de cada produto no item
item['modeSalesPrice'] =item['itemID'].map(get_mode_sales_price )


In [8]:
#colocando valor da moda no order
modeSalesPrice = item[['itemID','modeSalesPrice']]
order = pd.merge(order, modeSalesPrice, on='itemID')

print(order['modeSalesPrice'].describe().round(1))

count    2181955.0
mean          36.9
std          126.3
min            0.0
25%            7.2
50%           16.7
75%           34.6
max         9055.1
Name: modeSalesPrice, dtype: float64


In [9]:
print(order['modeSalesPrice'].describe().round(1))
print(order['modeSalesPrice'].isna().sum() )

count    2181955.0
mean          36.9
std          126.3
min            0.0
25%            7.2
50%           16.7
75%           34.6
max         9055.1
Name: modeSalesPrice, dtype: float64
0


In [10]:
print(len(order[order['salesPrice'] == order['modeSalesPrice']]))
print(len(order[order['salesPrice'] != order['modeSalesPrice']]))

1737973
443982


80% das vendas seguem o valor da moda

In [11]:
print(len(order[order['salesPrice'] < order['modeSalesPrice']]))

255515


11% das vendas tem valor abaixo da moda, poderiam ser considerados promoçoes (?);

In [12]:
print(len(order[order['salesPrice'] > order['modeSalesPrice']]))

188467


8% restantes que tem valor acima da moda, o que acho bizarro

### **adicionando a feature: diferença da moda com salesPrice**

In [13]:
order['difModa'] = order['salesPrice'] - order['modeSalesPrice']


In [14]:
print(order['difModa'].describe().round(1))
print(order['difModa'].isna().sum() )

count    2181955.0
mean          -0.2
std            9.3
min        -5141.5
25%            0.0
50%            0.0
75%            0.0
max          972.5
Name: difModa, dtype: float64
0


estou pressuponto que todo produto vendido abaixo da moda estava em promoç~ao, esta feature captura isso

tamb'em existe o contr'ario, seria algo como um super faturamento